## Normal Distribution and Gradient Descent

The **Normal Distribution**, also called the **Gaussian Distribution**, is the most commonly known distribution that produces a symmetrical bell curve. It is defined by a mean $ \mu $ and standard deviation $ \sigma $. It shows up frequently in nature, engineering, business, and even non-normal data  due to the central limit theorem. 

In this scenario we will use **maximum likelihoood estimation (MLE)** to find values for $ \mu $ and $ \sigma $ that best fits a normal distribution given some data. To execute the operation, we will use gradient descent with DiffKt to perform automatic differentiation for us. 

Let's bring in the DiffKt library for automatic differentiation and differentiable operations, as well as Lets-Plot for visualization.

In [1]:
%use lets-plot

In [2]:
@file:DependsOn("../kotlin/api/build/libs/api.jar")

The normal distribution is modeled with the following function. 

$$
f(x) = \frac{1}{\sigma \sqrt{2 \pi}}e^{-\frac{1}{2}(\frac{x-\mu}{\sigma})^2}
$$

This function below creates the symmetrical bell curve defined by a mean $ \mu $ and standard deviation $ \sigma $ as plotted below. In this case $ \mu $ (the mean) is 5 and $ \sigma $ (the standard deviation) is 1. Note on the line declaring the `xCurve` variable, we call `generateSequence(0f) { it + .01f }` to create a sequence of values starting at `0` and then incrementing repeatedly by `.01` to produce `0.0`, `0.01`, `0.02`, `0.03`, and so on. This would go on forever but we use `takeWhile { it <= 10f }` to stop the sequence when we reach `10`. We then use `toList()` to gather the generated values into a `List<Float>`. 

In [3]:
fun normalDistribution(x: Float, mean: Float, stdDev: Float) = 
     1.0F / (stdDev*(2f * kotlin.math.PI).pow(.5)) * exp(-.5f * ((x - mean) / stdDev).pow(2))

val xCurve = generateSequence(0f) { it + .01f }.takeWhile { it <= 10f }.toList()
val yCurve = xCurve.map { normalDistribution(it, 5f, 1f) }

letsPlot { x = xCurve; y = yCurve } + 
    ggsize(600, 500) + geomPoint(shape = 1, size = 1)

Now let's say we have some data on 50 golden retriever weights. Let's import and print this data. 

In [4]:
import java.net.URL

// Import golden retriever weights
val allData = URL("https://bit.ly/3ILpk6E")
    .readText().split(Regex("\\r?\\n"))
    .filter { it.isNotBlank() }
    .map { it.toFloat() }

    
print(allData)

[65.8, 67.7, 64.1, 69.3, 65.1, 60.8, 65.5, 64.6, 61.7, 64.2, 69.3, 57.4, 60.5, 68.0, 60.9, 64.4, 63.3, 59.8, 58.1, 63.6, 66.3, 61.3, 67.4, 62.2, 65.2, 67.3, 65.6, 62.3, 66.7, 60.1, 65.4, 66.0, 66.2, 63.6, 62.6, 64.0, 62.3, 67.5, 63.0, 67.9, 61.3, 65.1, 69.9, 66.1, 65.6, 62.2, 68.5, 65.9, 60.6, 69.5]

Here is what a fitted normal distribution looks like against the data. We will discuss how to get this fitted distribution shortly where $ \mu $ is $ 64.43 $ and the $ \sigma $ is $ 2.996 $. 

In [5]:
val xCurve = generateSequence(44f) { it + .01f }.takeWhile { it <= 84f }.toList()
val yCurve = xCurve.map { normalDistribution(it, 64.43407f, 2.9963064f) }

letsPlot { x = allData.plus(xCurve); y = generateSequence { 0F }.take(allData.count()).plus(yCurve).toList() } + 
    ggsize(600, 500) + geomPoint(shape = 1, size = 2)

How do we get this fit? We need to solve for the mean $ \mu $ and standard deviation $ \sigma $. While we could use the data to infer these two parameters using descriptive statistics, let's calculate them using maximum likelihood estimation. **Maximum likelihoood estimation (MLE)** is a technique that finds parameters most likely to produce the observed data, by maximizing the joint likelihood across all the data. In other words, for a given $ \mu $ and $ \sigma $ we calculate the likelihoods for each point using the normal distribution function, and then multiply them together.

$$
L(x) = \prod_{i=0}^{n}\frac{1}{\sigma \sqrt{2 \pi}}e^{-\frac{1}{2}(\frac{x_i-\mu}{\sigma})^2}
$$

Because we don't like multiplying decimals on computers due to floating point underflow issues, we are better off taking the `log()` or `ln()` of each likelihood and summing them rather than multiplying them. This circumvents the need to multiply decimals and floating point underflow issues, and perform this multiplication through logarithmic addition instead. 

$$
L(x) = \sum_{i=0}^{n}log(\frac{1}{\sigma \sqrt{2 \pi}}e^{-\frac{1}{2}(\frac{x_i-\mu}{\sigma})^2})
$$

After we took the log of the total likelihood function, we will still see the same minimums and maximums for each $ \mu $ and $ \sigma $ value. Below is a plot of total likelihoods across different $ \mu $ and $ \sigma $ values. The data actually follows a normal distribution where $ \mu = 0 $ and $ \sigma = 1 $, so notice that is where the total likelihood peaks.  

![](./resources/qBjeCIFocf.png)

For our golden retriever data, we can use gradient descent to maximize our total likelihood by changing $ \mu $ and $ \sigma $ in an iterative fashion following the gradients. Let's set our learning rate `lr` at `.01` and perform 100K iterations. We will initialize our mean $ \mu $ at one of the data points and the standard deviation $ \sigma $ at $ 1 $. While the gradient descent training will change these values, we need to initialize them somewhere so we will start them in the neighborhood of the data. 

Most importantly, we will use DiffKT's `reverseDerivatives()` function to calculate the gradients for $ \mu $ and $ \sigma $, which we then scale down with the learning rate  `lr` and subtract that from the coefficients respectively. 

In [9]:
import org.diffkt.*

// Create tensor of golden retriever weights
val dataTensor = tensorOf(*allData.toFloatArray())

// Initialize mean and standard deviation
// at one of the data points
var mean: DScalar = FloatScalar(allData[0])
var stdDev: DScalar = FloatScalar(1f)

// normal distribution function
fun normalDistribution(x: DTensor, mean: DScalar, stdDev: DScalar): DTensor =
    1f / (stdDev*(2f * FloatScalar.PI).pow(.5f)) * exp(-.5f * ((x - mean) / stdDev).pow(2))

fun mle(mean: DScalar, stdDev: DScalar) = ln(normalDistribution(dataTensor, mean, stdDev)).sum()

// perform gradient descent
val L = .05f
val iterations = 1000

for (i in 0..iterations) {
    val (meanGradient, stdDevGradient) = reverseDerivatives(mean, stdDev, ::mle)

    mean += L * meanGradient
    stdDev += L * stdDevGradient
}

print("mean=$mean, stdDev=$stdDev")

mean=64.43399f, stdDev=2.996306f

When you run gradient descent, you should get a mean of should get approximately mean of about `64.43399` and standard deviation of `2.996306`. When we plot this bell curve against our data, we see an appropriate fit to the data. 

In [11]:
val xCurve = generateSequence(44f) { it + .01f }
    .takeWhile { it <= 84f }
    .toList()
    
    
val yCurve = normalDistribution(tensorOf(*xCurve.toFloatArray()), mean, stdDev).let { yCurveTensor ->
    List(yCurveTensor.shape.product) { i -> (yCurveTensor as FloatTensor).at(i) }
}


letsPlot { 
    x = allData.plus(xCurve); 
    y = generateSequence { 0f }
        .take(allData.count())
        .plus(yCurve).toList() 
} + ggsize(600, 500) + geomPoint(shape = 1, size = 2)